In [1]:
import transformers
transformers.logging.set_verbosity_error()

In [2]:
import pandas as pd
from tqdm import tqdm
from caif.inference import inference
from transformers import AutoTokenizer
from russian_paraphrasers.utils import clean

In [3]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

In [4]:
#constants
lm_checkpoint = 'alenusch/rugpt3-paraphraser'
cls_checkpoint = 'BunnyNoBugs/rubert-tiny2-russe-toxicity'

In [5]:
tokenizer = AutoTokenizer.from_pretrained(lm_checkpoint)

In [6]:
sentence = 'я хочу тебя убить нахуй блядь пидорас сука'
prompt = f'{sentence} === '

In [7]:
max_length = int(len(tokenizer.encode(prompt)) * 1.5 + 10)

In [9]:
result = inference(
    lm_model_name=lm_checkpoint,
    cls_model_name=cls_checkpoint,
    prompt=prompt,
    fp16=True,
    alpha=10,
    target_label_id=1,
    entropy_threshold=0,
    act_type='sigmoid',
    num_tokens=30
)

In [10]:
def caif_paraphrase(
        text,
        prompt_format,
        tokenizer,
        lm_model_name,
        cls_model_name,
        fp16=True,
        alpha=-5,
        target_label_id=1,
        entropy_threshold=0,
        act_type='sigmoid',
        stop_token='</s>'
):
    prompt = prompt_format.format(text)
    max_length = int(len(tokenizer.encode(prompt)) * 1.5 + 10)
    output_text = inference(
        lm_model_name=lm_model_name,
        cls_model_name=cls_model_name,
        prompt=prompt,
        fp16=fp16,
        alpha=alpha,
        target_label_id=target_label_id,
        entropy_threshold=entropy_threshold,
        act_type=act_type,
        num_tokens=max_length
    )
    output_text = output_text[: output_text.find(stop_token) if stop_token else None]
    output_text = output_text[len(prompt):]
    output_text = clean(output_text)

    return output_text

In [11]:
caif_paraphrase(
    text='вы чо курите блять ?',
    prompt_format='{} === ',
    tokenizer=tokenizer,
    lm_model_name=lm_checkpoint,
    cls_model_name=cls_checkpoint
)

'У вас чо за хуйня с башкой'

In [13]:
para_results = []
for i in tqdm(toxic_inputs):
    para_result = caif_paraphrase(
        text=i,
        prompt_format='{} === ',
        tokenizer=tokenizer,
        lm_model_name=lm_checkpoint,
        cls_model_name=cls_checkpoint,
        fp16=True,
        alpha=-5,
        target_label_id=1,
        entropy_threshold=1.0,
    )
    print(para_result)
    para_results.append(para_result)

  0%|          | 0/800 [00:10<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.